#git clone

In [1]:
!git clone https://github.com/LeQuangHuyUIT/Reinforcement-learning-tetris

Cloning into 'Reinforcement-learning-tetris'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), done.


In [2]:
!pip install pygame

     |████████████████████████████████| 18.3 MB 1.4 MB/s 


In [3]:
%cd /content/Reinforcement-learning-tetris/src

/content/Reinforcement-learning-tetris/src


# import lib

In [4]:
import random
import heapq
from numpy.linalg import cholesky
import numpy as np
from collections import deque
from tetris_utils import *
from tetris_reinforcement_learner import TetrisReinforcementLearner
from tetris import *
from copy import deepcopy
import pygame,sys
import os
import matplotlib.pyplot as plt
from tetris import TetrisApp
from tetris_reinforcement_learner import TetrisReinforcementLearner
import math

pygame 2.1.0 (SDL 2.0.16, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Init

In [9]:
App=TetrisApp()
RL_learner=TetrisReinforcementLearner(App)
done = False
batch_size = 40


state_size = 270
action_size = 30
# agent.load("./save/cartpole-dqn.h5")
App.init_game()
def pixalize_stone(stone):
    container=np.zeros((2,10))
    if len(stone)==1:
        container[0][...,3:3+len(stone[0])]=stone[0]
    elif stone[0][0]==7:
        container[0][...,4:4+len(stone[0])]=stone[0]
        container[1][...,4:4+len(stone[0])]=stone[1]
    else:
        container[0][...,3:3+len(stone[0])]=stone[0]
        container[1][...,3:3+len(stone[0])]=stone[1]
    return container
    
def preprocess_data(state):
    
    board=np.array(state['board'])
    stone=state['stone']
    next_stone=state['next_stone']
    pixal_stone=pixalize_stone(stone)
    pixal_next_stone=pixalize_stone(next_stone)

    final_board=np.append(pixal_next_stone,pixal_stone,axis=0)
    final_board=np.append(final_board,board,axis=0)
    final_board=np.where(final_board>0,1,0)
#     print(final_board)
    final_board=final_board.flatten()
    final_board=final_board.reshape(1,270)
    return final_board


def update_new(reinforcement_learner, old_state,new_state):## Taking holes into account
    change_in_pile_height = reinforcement_learner.get_pile_height(new_state) - reinforcement_learner.get_pile_height(old_state)
    reward = -1 * change_in_pile_height
    return reward

def get_feature(state):
    height=RL_learner.get_pile_height(state)
    holes=RL_learner.get_holes(state)
    contours=RL_learner.get_contours(state)
    row_holes=RL_learner.get_row_holes(state)
    features=np.array([[height],[holes],[contours],[row_holes]])
    return features
    
def get_current_state_CEM_value(state,action_sequence):
    successor_state=RL_learner.get_successor_state(state,action_sequence)
    features=get_feature(successor_state)
    for feature, value in features.items():
        CEM_val+=RL_learner.CEM_weights[feature]*value
        
def get_following_state_CEM_value(state,action_sequence,cur_theta):
    
    successor_state=RL_learner.get_successor_state(state,action_sequence)
    next_state_reward=RL_learner.get_next_action_reward(state,action_sequence)
#     print('next_state_reward is',next_state_reward)
    done=successor_state['gameover']
    if not done:
        
        features=get_feature(successor_state)
        features=np.append(features,np.array([[next_state_reward]]),axis=0)
    return cur_theta.T.dot(features)

def get_sample_theta(mu,sigma,num_sample):
    R = cholesky(sigma)
    s = np.dot(np.random.randn(num_sample, 5), R) + mu.T
    return s

In [25]:
def evaluate(cur_theta):
    App.init_game()
    # if not App.pygame_initted: App.init_pygame()
    App.gameover=False
    App.paused=False
    App.start_game()
    done=False

    cur_theta=np.array(cur_theta)
    cur_theta=cur_theta[:,np.newaxis]

    while not done:
        state=RL_learner.capture_state_attributes(App)

        ## calculate the next state value, which is store in the list: next_val
        next_val=[] # store the next_state_value
        legal_action_sequences = RL_learner.get_legal_action_sequences(state)

        for actions in legal_action_sequences:
            next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
        next_action=legal_action_sequences[np.argmax(next_val)]
        
        App.play_action_sequence(next_action)
        new_state=RL_learner.capture_state_attributes(App)
        done=new_state['gameover']
        if done:
            break
    
    return App.lines

In [16]:
def cal_mu(weights, Z):
    return sum([w * z for w, z in zip(weights, Z)])

# Code CMA-ES


In [17]:
from math import floor, log, sqrt

import torch
from torch import nn as nn

In [18]:
def expect_multivariate_norm(N):
    return N ** 0.5 * (1 - 1 / (4 * N) + 1 / (21 * N ** 2))
    
def sample(n, sigma, mean, B, D):
    features = mean.size(0)
    z = torch.randn(features, n, device=mean.device, dtype=mean.dtype)
    s = mean.view(-1, 1) + sigma * B.matmul(D.matmul(z))
    return s.T, z.T

In [36]:
def objective_f(s):
    thetas = s.detach().cpu().numpy()
    return [evaluate(theta) for theta in thetas]

In [45]:
class CMA_ES():
    def __init__(self, N, step_mode='auto', step_decay=None, initial_step_size=None, samples=None, oversample=0.0):
        self.N = N
        self.recommended_steps = range(1, floor(1e3 * N ** 2))

        # selection settings
        self.samples = 4 + floor(3 * log(N)) if samples is None else samples
        self.oversample = oversample
        self.mu = self.samples / 2
        self.weights = torch.tensor([log(self.mu + 0.5)]) - torch.linspace(start=1, end=self.mu,
                                                                           steps=floor(self.mu)).log()
        self.weights = self.weights / self.weights.sum()
        self.weights = self.weights / self.weights.sum()
        self.mu = floor(self.mu)
        self.mueff = (self.weights.sum() ** 2 / (self.weights ** 2).sum()).item()

        # adaptation settings
        self.cc = (4 + self.mueff / N) / (N + 4 + 2 * self.mueff / N)
        self.cs = (self.mueff + 2) / (N + self.mueff + 5)
        self.c1 = 2 / ((self.N + 1.3) ** 2 + self.mueff)
        self.cmu = 2 * (self.mueff - 2 + 1 / self.mueff) / ((N + 2) ** 2 + 2 * self.mueff / 2)
        self.damps = 1 + 2 * max(0.0, sqrt((self.mueff - 1.0) / (N + 1)) - 1) + self.cs
        self.chiN = expect_multivariate_norm(N)
        self.step_size = 0.5 if initial_step_size is None else initial_step_size
        self.step_mode = step_mode
        if step_mode == 'decay' and step_decay is None:
            raise Exception('decay mode requires you set a step decay')
        self.step_decay = (1.0 - step_decay) if step_decay is not None else None

        # variables
        self.mean = torch.zeros(N)
        self.b = torch.eye(N)
        self.d = torch.eye(N)
        self.c = torch.matmul(self.b.matmul(self.d), self.b.matmul(self.d).T)  # c = B D D B.T

        self.pc = torch.zeros(N)
        self.ps = torch.zeros(N)
        self.gen_count = 1

    def step(self, objective_f):
        s, z = sample(self.samples + floor(self.oversample), self.step_size, self.mean, self.b, self.d)
        self.oversample = self.oversample * 0.8

        # print(s, z)
        f = objective_f(s)
        results = [{'parameters': s[i], 'z': z[i], 'fitness': f} for i, f in enumerate(f)]

        ranked_results = sorted(results, key=lambda x: x['fitness'], reverse=True)

        selected_results = ranked_results[0:self.mu]
        z = torch.stack([g['z'] for g in selected_results])
        g = torch.stack([g['parameters'] for g in selected_results])

        self.mean = (g * self.weights.unsqueeze(1)).sum(0)
        zmean = (z * self.weights.unsqueeze(1)).sum(0)

         # step size
        self.ps = (1 - self.cs) * self.ps + sqrt(self.cs * (2.0 - self.cs)) * self.b.matmul(zmean)

        correlation = self.ps.norm() / self.chiN

        # delay the introduction of the rank 1 update
        denominator = sqrt(1 - (1 - self.cs) ** (2 * self.gen_count / self.samples))
        threshold = 1.4e2 / self.N + 1
        hsig = correlation / denominator < threshold
        hsig = 1.0 if hsig else 0.0

        # adapt step size
        if self.step_mode == 'auto':
            self.step_size = self.step_size * ((self.cs / self.damps) * (correlation - 1.0)).exp()
        elif self.step_mode == 'nodamp':
            self.step_size = self.step_size * (correlation - 1.0).exp()
        elif self.step_mode == 'decay':
            self.step_size = self.step_size * self.step_decay
        elif self.step_mode == 'constant':
            pass
        else:
            raise Exception('step_mode must be auto | nodamp | decay | constant')

        # a mind bending way to write a exponential smoothed moving average
        # zmean does not contain step size or mean, so allows us to add together
        # updates of different step sizes
        self.pc = (1 - self.cc) * self.pc + hsig * sqrt(self.cc * (2.0 - self.cc) * self.mueff) * self.b.matmul(
            self.d).matmul(zmean)
        # which we then combine to make a covariance matrix, from 1 (mean) datapoint!
        # this is why it's called "rank 1" update
        pc_cov = self.pc.unsqueeze(1).matmul(self.pc.unsqueeze(1).t())
        # mix back in the old covariance if hsig == 0
        pc_cov = pc_cov + (1 - hsig) * self.cc * (2 - self.cc) * self.c

        # estimate cov for all selected samples (weighted by rank)
        bdz = self.b.matmul(self.d).matmul(z.t())
        cmu_cov = torch.matmul(bdz, self.weights.diag_embed())
        cmu_cov = cmu_cov.matmul(bdz.t())

        self.c = (1.0 - self.c1 - self.cmu) * self.c + (self.c1 * pc_cov) + (self.cmu * cmu_cov)

        # pull out the eigenthings and do the business
        self.d, self.b = torch.symeig(self.c, eigenvectors=True)
        self.d = self.d.sqrt().diag_embed()
        self.gen_count += 1

        info = {'step_size': self.step_size, 'correlation': correlation,
                'fitness_max': max(f), 'fitness_mean': np.mean(f), 'c_norm': self.c.norm(),
                'max_eigv': self.d.max()}
        return ranked_results, info

In [ ]:
cma_es = CMA_ES(5, samples= 50, oversample= 0.5)

for i in range(1000):
    ranked_results, info= cma_es.step(objective_f)
    print(info)

{'step_size': tensor(0.4454), 'correlation': tensor(0.5476), 'fitness_max': 538, 'fitness_mean': 18.26, 'c_norm': tensor(2.8617), 'max_eigv': tensor(1.4884)}
{'step_size': tensor(0.3938), 'correlation': tensor(0.5188), 'fitness_max': 310, 'fitness_mean': 28.04, 'c_norm': tensor(3.5355), 'max_eigv': tensor(1.7580)}
{'step_size': tensor(0.3426), 'correlation': tensor(0.4551), 'fitness_max': 575, 'fitness_mean': 65.62, 'c_norm': tensor(4.0270), 'max_eigv': tensor(1.9242)}
{'step_size': tensor(0.2956), 'correlation': tensor(0.4227), 'fitness_max': 1126, 'fitness_mean': 139.42, 'c_norm': tensor(4.7799), 'max_eigv': tensor(2.1408)}
